In [1]:
import os
from pathlib import Path

testfolder = 'Simulations/1axis'

if not os.path.exists(testfolder):
    os.makedirs(testfolder)
    
print ("Your simulation will be stored in %s" % testfolder)

Your simulation will be stored in Simulations/1axis


In [2]:
import bifacial_radiance as br
import bifacialvf as bf
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [3]:
# This information helps with debugging and getting support :)
import sys, platform
print("Working on a ", platform.system(), platform.release())
print("Python version ", sys.version)
print("Pandas version ", pd.__version__)
print("bifacial_radiance version ", br.__version__)
print("pyplot ", plt.matplotlib.__version__)
print("bifacialvf version ", bf.__version__)

Working on a  Darwin 21.6.0
Python version  3.11.5 (main, Sep 11 2023, 08:19:27) [Clang 14.0.6 ]
Pandas version  2.0.3
bifacial_radiance version  0.4.2+241.gef55bdd.dirty
pyplot  3.7.2
bifacialvf version  0.1.8.1


In [4]:
simulationname = 'bifacialvf_1axis'

# Location
lat = 39.7555
lon = -105.2211

# Scene Parameters
azimuth_ang=90 # Facing south
tilt=30

# MakeModule Parameters
moduletype='PVmod'
numpanels=1
module_x = 2 # m
module_y = 1 # m. slope we will measure
sensorsy=2
sensorsground=5

# SceneDict Parameters
pitch = 6 # m
albedo = 0.2
clearance_height = 0.5 # m  
nMods = 4 
nRows = 3

sceneDict = {'tilt':10,'pitch':3,'clearance_height':0.2,'azimuth':180, 'nMods': 20, 'nRows': 7} 

In [5]:
demo = br.RadianceObj(simulationname,path = testfolder)
demo.setGround(albedo)
epwfile = demo.getEPW(lat, lon)

path = Simulations/1axis
Loading albedo, 1 value(s), 0.200 avg
1 nonzero albedo values.
Getting weather file: USA_CO_Golden-NREL.724666_TMY3.epw
 ... OK!


In [6]:
module=demo.makeModule(name=moduletype,x=module_x,y=module_y)


Module Name: PVmod
Module PVmod updated in module.json
Pre-existing .rad file objects/PVmod.rad will be overwritten



In [7]:
hub_height = 1.5
pitch = 5.7
sazm = 180 # axis angle, N-S = 180
fixed_tilt_angle = None
gcr = 2 / pitch
cumulativesky = True

trackerParams = {
    'limit_angle': 50,
    'angledelta': 30,
    'backtrack': True,
    'gcr': gcr,
    'cumulativesky': cumulativesky,
    'azimuth': sazm,  # axis angle, N-S = 180
    'fixed_tilt_angle': fixed_tilt_angle
}

In [8]:
metdata = demo.readWeatherFile(epwfile, coerce_year=2021, starttime='2021-06-01', endtime='2021-06-30_23')

8760 line in WeatherFile. Assuming this is a standard hourly WeatherFile for the year for purposes of saving Gencumulativesky temporary weather files in EPW folder.
Coercing year to 2021
Filtering dates
Saving file EPWs/metdata_temp.csv, # points: 8760
Calculating Sun position for Metdata that is right-labeled  with a delta of -30 mins. i.e. 12 is 11:30 sunpos


In [9]:
trackerdict = demo.set1axis(**trackerParams)

Saving file EPWs/1axis_-60.0.csv, # points: 90
Saving file EPWs/1axis_-30.0.csv, # points: 90
Saving file EPWs/1axis_-0.0.csv, # points: 90
Saving file EPWs/1axis_30.0.csv, # points: 90
Saving file EPWs/1axis_60.0.csv, # points: 90


In [10]:
trackerdict

{-0.0: {'csvfile': 'EPWs/1axis_-0.0.csv',
  'surf_azm': 90.00000409455455,
  'surf_tilt': 0.0,
  'datetime': Index(['2021-06-01 05:00:00', '2021-06-01 12:00:00', '2021-06-01 13:00:00',
         '2021-06-02 05:00:00', '2021-06-02 12:00:00', '2021-06-02 13:00:00',
         '2021-06-03 05:00:00', '2021-06-03 12:00:00', '2021-06-03 13:00:00',
         '2021-06-04 05:00:00', '2021-06-04 12:00:00', '2021-06-04 13:00:00',
         '2021-06-05 05:00:00', '2021-06-05 12:00:00', '2021-06-05 13:00:00',
         '2021-06-06 05:00:00', '2021-06-06 12:00:00', '2021-06-06 13:00:00',
         '2021-06-07 05:00:00', '2021-06-07 12:00:00', '2021-06-07 13:00:00',
         '2021-06-08 05:00:00', '2021-06-08 12:00:00', '2021-06-08 13:00:00',
         '2021-06-09 05:00:00', '2021-06-09 12:00:00', '2021-06-09 13:00:00',
         '2021-06-10 05:00:00', '2021-06-10 12:00:00', '2021-06-10 13:00:00',
         '2021-06-11 05:00:00', '2021-06-11 12:00:00', '2021-06-11 13:00:00',
         '2021-06-12 05:00:00', '20

In [12]:
sceneDict = {
    'pitch': pitch,
    'hub_height': hub_height,
    'nMods': 5,
    'nRows': 2
}

In [13]:
trackerdict = demo.makeScene1axis(module=moduletype, sceneDict=sceneDict)
trackerdict = demo.makeOct1axis()


Making .rad files for cumulativesky 1-axis workflow
5 Radfiles created in /objects/

Making 5 octfiles in root directory.
Created 1axis_-60.0.oct
Created 1axis_-30.0.oct
Created 1axis_-0.0.oct
Created 1axis_30.0.oct
Created 1axis_60.0.oct


In [14]:
trackerdict.keys()

dict_keys([-0.0, -30.0, -60.0, 30.0, 60.0])

In [16]:
trackerdict

{-0.0: {'csvfile': 'EPWs/1axis_-0.0.csv',
  'surf_azm': 90.00000409455455,
  'surf_tilt': 0.0,
  'datetime': Index(['2021-06-01 05:00:00', '2021-06-01 12:00:00', '2021-06-01 13:00:00',
         '2021-06-02 05:00:00', '2021-06-02 12:00:00', '2021-06-02 13:00:00',
         '2021-06-03 05:00:00', '2021-06-03 12:00:00', '2021-06-03 13:00:00',
         '2021-06-04 05:00:00', '2021-06-04 12:00:00', '2021-06-04 13:00:00',
         '2021-06-05 05:00:00', '2021-06-05 12:00:00', '2021-06-05 13:00:00',
         '2021-06-06 05:00:00', '2021-06-06 12:00:00', '2021-06-06 13:00:00',
         '2021-06-07 05:00:00', '2021-06-07 12:00:00', '2021-06-07 13:00:00',
         '2021-06-08 05:00:00', '2021-06-08 12:00:00', '2021-06-08 13:00:00',
         '2021-06-09 05:00:00', '2021-06-09 12:00:00', '2021-06-09 13:00:00',
         '2021-06-10 05:00:00', '2021-06-10 12:00:00', '2021-06-10 13:00:00',
         '2021-06-11 05:00:00', '2021-06-11 12:00:00', '2021-06-11 13:00:00',
         '2021-06-12 05:00:00', '20

In [17]:
trackerdict = demo.runViewFactor(sensorsy=4, backtrack=trackerParams['backtrack'], limit_angle=trackerParams['limit_angle'])

TypeError: unsupported operand type(s) for /: 'NoneType' and 'float'